# 🎙️ Fine-tune viXTTS on Vietnamese VietSpeech Dataset

This notebook fine-tunes the **viXTTS** (Vietnamese XTTS) model on the **VietSpeech** dataset.

## 📋 Overview

- **Base Model**: [capleaf/viXTTS](https://huggingface.co/capleaf/viXTTS) - Vietnamese adaptation of XTTS-v2
- **Dataset**: [NhutP/VietSpeech](https://huggingface.co/datasets/NhutP/VietSpeech) - Vietnamese speech dataset
- **Framework**: Coqui TTS 0.27.2
- **Requirements**: Python 3.13, CUDA GPU (T4 or better)

## ⚙️ System Requirements

- **GPU**: T4 (15GB VRAM) or better
- **Storage**: ~30-50GB free space
- **RAM**: 12GB+ recommended
- **Python**: 3.13

## 🚀 Quick Start

Run cells in order from top to bottom. The notebook is organized into sections:

1. **Environment Setup**
2. **Configuration**
3. **Dataset Preparation**
4. **Training**
5. **Inference**

---

# SECTION 1: Environment Setup

- Mount Google Drive
- Set HF Token
- Set Memory Management Environment Variables
- Install system dependencies
- Install Coqui TTS
- Download viXTTS Base Model
- Check GPU
- Check Storage
- Verify package installation, verify transformers and TTS can load GPT2 models

In [2]:
# Mount Google Drive

from google.colab import drive
import os

# Mount Drive for persistent storage
drive.mount('/content/drive')

# Define directories
RUN_DIR = "/content/drive/MyDrive/vixtts_runs/vietspeech_run1"
DATA_DIR = "/content/datasets"
MAN_DIR = f"{DATA_DIR}/manifests_vietspeech"

# Create directories
!mkdir -p "$RUN_DIR" "$DATA_DIR" "$MAN_DIR"

print(f"✅ Directories created:")
print(f"   RUN_DIR: {RUN_DIR}")
print(f"   DATA_DIR: {DATA_DIR}")

Mounted at /content/drive
✅ Directories created:
   RUN_DIR: /content/drive/MyDrive/vixtts_runs/vietspeech_run1
   DATA_DIR: /content/datasets


In [3]:
# Set HF Token
import os
from google.colab import userdata

try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("✅ HF_TOKEN loaded from Colab Secrets")
except:
    print("⚠️  Failed to load HF_TOKEN token. Please use Colab Secrets!")
    HF_TOKEN = "hf_YOUR_TOKEN_HERE"

os.environ["HF_TOKEN"] = HF_TOKEN

✅ HF_TOKEN loaded from Colab Secrets


In [4]:
# CRITICAL: Set Memory Management Environment Variables
# Must run BEFORE installing/importing any libraries!

import os

print("🔧 Configuring memory management for 12GB RAM system...")

# Force datasets library to use PyTorch backend (prevents TF from loading)
os.environ["HF_DATASETS_OFFLINE"] = "0"  # Allow downloads
os.environ["DISABLE_TELEMETRY"] = "1"

# TensorFlow memory management (in case it gets loaded)
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"  # Don't allocate all GPU memory upfront
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"  # Use async allocator
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # Reduce logging
os.environ["TF_CPP_MAX_VLOG_LEVEL"] = "0"  # Suppress verbose logging

# XLA (used by TensorFlow/JAX) memory optimization
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"  # Don't preallocate memory
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"  # Use platform allocator
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.25"  # Limit to 25% of GPU memory

# PyTorch memory optimization
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512,expandable_segments:True"

# CUDA conflict suppression (fixes duplicate registration warnings)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_MODULE_LOADING"] = "LAZY"  # Lazy load CUDA modules to avoid conflicts

# NumPy memory management
os.environ["OMP_NUM_THREADS"] = "2"  # Limit CPU threads
os.environ["MKL_NUM_THREADS"] = "2"

print("✅ Memory management environment configured:")
print("   - TensorFlow GPU growth: enabled")
print("   - XLA preallocation: disabled")
print("   - PyTorch CUDA: optimized")
print("   - CUDA conflicts: suppressed")
print("   - CPU threads: limited to 2")


🔧 Configuring memory management for 12GB RAM system...
✅ Memory management environment configured:
   - TensorFlow GPU growth: enabled
   - XLA preallocation: disabled
   - PyTorch CUDA: optimized
   - CUDA conflicts: suppressed
   - CPU threads: limited to 2


In [5]:
# Install system dependencies
%%capture
!apt-get update -q
!apt-get install -y -qq ffmpeg
print("✅ FFmpeg installed")


In [6]:
# Install Python Dependencies

print("📦 Installing PyTorch and dependencies")

print("🧹 Cleaning conflicting vision packages (torchvision, fastai, timm)...")
# These pull in torchvision and cause the circular import error when TTS imports
!pip uninstall -y torchvision fastai timm

print("📦 Installing PyTorch (GPU) + torchaudio for CUDA 12.x...")
!pip install -q \
"torch==2.4.1" "torchvision==0.19.1" "torchaudio==2.4.1" \
--index-url https://download.pytorch.org/whl/cu124

print("📦 Installing datasets, librosa, soundfile")
!pip install -q datasets librosa soundfile

print("✅ PyTorch installed")

📦 Installing PyTorch and dependencies
🧹 Cleaning conflicting vision packages (torchvision, fastai, timm)...
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: fastai 2.8.5
Uninstalling fastai-2.8.5:
  Successfully uninstalled fastai-2.8.5
Found existing installation: timm 1.0.22
Uninstalling timm-1.0.22:
  Successfully uninstalled timm-1.0.22
📦 Installing PyTorch (GPU) + torchaudio for CUDA 12.x...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 875.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 121.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 110.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.4/883.4 kB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.2 MB/s eta

In [7]:
# Install missing faster_whisper dependency
print("📦 Installing faster_whisper (required for XTTS training)...")
%pip install -q faster-whisper
print("✅ faster_whisper installed\n")

📦 Installing faster_whisper (required for XTTS training)...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.2 MB/s eta 0:00:00
✅ faster_whisper installed



In [8]:
# Install Coqui TTS
print("📦 Installing transformers + Coqui TTS 0.27.2...")

# transformers 4.55.4 is fine and already works with GPT2Model imports
!pip install -q "transformers==4.55.4" "coqui-tts==0.27.2"

print("✅ Coqui TTS 0.27.2")

📦 Installing transformers + Coqui TTS 0.27.2...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 41.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 49.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 60.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata 

In [9]:
# =============================================================================
# 🔄 SMART CHECKPOINT RESOLUTION (Enhanced)
# =============================================================================
# Priority: 1) Existing fine-tuned checkpoint, 2) Cached base model, 3) Download fresh
# Features:
#   - Validates checkpoint integrity with PyTorch
#   - Lists all available checkpoints
#   - Supports multiple run patterns (GPT_XTTS_FT-*, vixtts_*)
#   - Force fresh start option
# =============================================================================

import glob
import os
import re
import shutil
from datetime import datetime
from huggingface_hub import snapshot_download, try_to_load_from_cache
from huggingface_hub.utils import LocalEntryNotFoundError

print("🔍 Smart Checkpoint Resolution")
print("=" * 60)

# =============================================================================
# CONFIGURATION
# =============================================================================
TRAINING_OUTPUT_DIR = f"{RUN_DIR}/run/training"
ORIGINAL_MODEL_DIR = f"{TRAINING_OUTPUT_DIR}/XTTS_v2.0_original_model_files"
HF_REPO_ID = "capleaf/viXTTS"

# Set to True to ignore existing checkpoints and start fresh
FORCE_FRESH_START = False

# Minimum checkpoint size in MB (smaller files are likely corrupted)
MIN_CHECKPOINT_SIZE_MB = 100

# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def get_checkpoint_info(checkpoint_path):
    """Get detailed info about a checkpoint file."""
    if not os.path.exists(checkpoint_path):
        return None
    
    size_mb = os.path.getsize(checkpoint_path) / (1024 * 1024)
    mtime = os.path.getmtime(checkpoint_path)
    date_str = datetime.fromtimestamp(mtime).strftime("%Y-%m-%d %H:%M")
    
    # Extract step/epoch number from filename
    basename = os.path.basename(checkpoint_path)
    step_match = re.search(r'(\d+)', basename)
    step = int(step_match.group(1)) if step_match else 0
    
    return {
        "path": checkpoint_path,
        "basename": basename,
        "size_mb": size_mb,
        "date": date_str,
        "mtime": mtime,
        "step": step,
        "is_best": "best_model" in basename.lower()
    }

def validate_checkpoint(checkpoint_path, quick_check=True):
    """Validate checkpoint integrity by trying to load it with PyTorch."""
    try:
        import torch
        if quick_check:
            # Quick check: just verify it's a valid torch file
            with open(checkpoint_path, 'rb') as f:
                # Read magic number (PyTorch files start with specific bytes)
                magic = f.read(8)
                # PyTorch pickle files or zip files (for newer format)
                is_valid = magic[:2] == b'\x80\x02' or magic[:2] == b'PK'
            return is_valid
        else:
            # Full check: actually load the checkpoint (slower but thorough)
            checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=False)
            has_model = 'model' in checkpoint or 'state_dict' in checkpoint
            return has_model
    except Exception as e:
        print(f"   ⚠️  Validation failed for {os.path.basename(checkpoint_path)}: {e}")
        return False

def find_all_checkpoints():
    """Find all checkpoints from all training runs."""
    if not os.path.exists(TRAINING_OUTPUT_DIR):
        return []
    
    # Look for multiple run directory patterns
    run_patterns = [
        f"{TRAINING_OUTPUT_DIR}/GPT_XTTS_FT-*",      # Default TTS trainer pattern
        f"{TRAINING_OUTPUT_DIR}/vixtts_*",           # Custom vixtts runs
        f"{TRAINING_OUTPUT_DIR}/XTTS_*",             # Generic XTTS runs
    ]
    
    all_run_dirs = []
    for pattern in run_patterns:
        all_run_dirs.extend(glob.glob(pattern))
    
    # Remove duplicates and sort by modification time (newest first)
    all_run_dirs = list(set(all_run_dirs))
    all_run_dirs.sort(key=lambda x: os.path.getmtime(x) if os.path.exists(x) else 0, reverse=True)
    
    all_checkpoints = []
    for run_dir in all_run_dirs:
        # Find all checkpoint files
        checkpoint_patterns = [
            f"{run_dir}/best_model*.pth",
            f"{run_dir}/checkpoint_*.pth",
            f"{run_dir}/model*.pth",
        ]
        
        for pattern in checkpoint_patterns:
            for ckpt_path in glob.glob(pattern):
                info = get_checkpoint_info(ckpt_path)
                if info and info["size_mb"] >= MIN_CHECKPOINT_SIZE_MB:
                    info["run_dir"] = run_dir
                    info["run_name"] = os.path.basename(run_dir)
                    all_checkpoints.append(info)
    
    # Sort: best_model first, then by modification time (newest first)
    all_checkpoints.sort(key=lambda x: (x["is_best"], x["mtime"]), reverse=True)
    
    return all_checkpoints

def find_original_model_files():
    """Check if original model files exist (for vocab, dvae, mel_stats)."""
    if not os.path.exists(ORIGINAL_MODEL_DIR):
        return None
    
    required = ["config.json", "vocab.json", "dvae.pth", "mel_stats.pth", "model.pth"]
    missing = [f for f in required if not os.path.exists(f"{ORIGINAL_MODEL_DIR}/{f}")]
    
    if missing:
        print(f"   ⚠️  Original model missing files: {missing}")
        return None
    
    return ORIGINAL_MODEL_DIR

def get_cached_base_model():
    """Check if base model is already in HuggingFace cache."""
    try:
        cached_path = try_to_load_from_cache(
            repo_id=HF_REPO_ID,
            filename="config.json"
        )
        if cached_path and os.path.exists(cached_path):
            return os.path.dirname(cached_path)
    except (LocalEntryNotFoundError, Exception):
        pass
    return None

def download_and_cache_base_model():
    """Download base model from HuggingFace and cache to persistent storage."""
    print(f"\n⬇️  Downloading base model from HuggingFace ({HF_REPO_ID})...")
    
    downloaded_path = snapshot_download(
        repo_id=HF_REPO_ID,
        token=os.environ.get("HF_TOKEN")
    )
    print(f"   ✅ Downloaded to cache: {downloaded_path}")
    
    # Copy to persistent storage on Google Drive
    if not os.path.exists(ORIGINAL_MODEL_DIR):
        print(f"\n📦 Copying to persistent storage...")
        os.makedirs(ORIGINAL_MODEL_DIR, exist_ok=True)
        
        for f in os.listdir(downloaded_path):
            src = f"{downloaded_path}/{f}"
            dst = f"{ORIGINAL_MODEL_DIR}/{f}"
            if os.path.isfile(src):
                shutil.copy2(src, dst)
                print(f"   ✅ {f}")
        
        print(f"   📁 Saved to: {ORIGINAL_MODEL_DIR}")
    
    return downloaded_path

# =============================================================================
# CHECKPOINT RESOLUTION
# =============================================================================

RESUME_FROM_CHECKPOINT = None
BASE_INIT = None
CHECKPOINT_SOURCE = None
ALL_AVAILABLE_CHECKPOINTS = []

# Step 1: Find all available checkpoints
print("\n📂 Scanning for existing checkpoints...")
ALL_AVAILABLE_CHECKPOINTS = find_all_checkpoints()

if ALL_AVAILABLE_CHECKPOINTS:
    print(f"\n✅ Found {len(ALL_AVAILABLE_CHECKPOINTS)} checkpoint(s):\n")
    print(f"{'#':<3} {'Type':<12} {'Size':<10} {'Date':<18} {'Run'}")
    print("-" * 80)
    
    for i, ckpt in enumerate(ALL_AVAILABLE_CHECKPOINTS[:10]):  # Show top 10
        ckpt_type = "⭐ best" if ckpt["is_best"] else "   ckpt"
        size_str = f"{ckpt['size_mb']:.0f} MB"
        print(f"{i+1:<3} {ckpt_type:<12} {size_str:<10} {ckpt['date']:<18} {ckpt['run_name'][:30]}")
    
    if len(ALL_AVAILABLE_CHECKPOINTS) > 10:
        print(f"   ... and {len(ALL_AVAILABLE_CHECKPOINTS) - 10} more")
else:
    print("   No existing checkpoints found")

# Step 2: Select best checkpoint (unless forcing fresh start)
if ALL_AVAILABLE_CHECKPOINTS and not FORCE_FRESH_START:
    best_ckpt = ALL_AVAILABLE_CHECKPOINTS[0]
    
    print(f"\n🔍 Validating best checkpoint: {best_ckpt['basename']}...")
    if validate_checkpoint(best_ckpt["path"], quick_check=True):
        print(f"   ✅ Checkpoint is valid!")
        RESUME_FROM_CHECKPOINT = best_ckpt["path"]
        CHECKPOINT_SOURCE = "finetuned"
    else:
        print(f"   ❌ Checkpoint validation failed, trying next...")
        # Try other checkpoints
        for ckpt in ALL_AVAILABLE_CHECKPOINTS[1:5]:  # Try up to 5
            if validate_checkpoint(ckpt["path"], quick_check=True):
                print(f"   ✅ Found valid checkpoint: {ckpt['basename']}")
                RESUME_FROM_CHECKPOINT = ckpt["path"]
                CHECKPOINT_SOURCE = "finetuned"
                break

elif FORCE_FRESH_START:
    print("\n⚠️  FORCE_FRESH_START=True: Ignoring existing checkpoints")

# Step 3: Find or download base model files (always needed for vocab, dvae, etc.)
print("\n📂 Locating base model files (vocab, dvae, mel_stats)...")

original_files = find_original_model_files()
if original_files:
    print(f"   ✅ Found in persistent storage: {original_files}")
    BASE_INIT = original_files
else:
    # Try HuggingFace cache
    cached_base = get_cached_base_model()
    if cached_base:
        print(f"   ✅ Found in HuggingFace cache: {cached_base}")
        BASE_INIT = cached_base
        if not CHECKPOINT_SOURCE:
            CHECKPOINT_SOURCE = "cached_base"
    else:
        # Download fresh
        BASE_INIT = download_and_cache_base_model()
        if not CHECKPOINT_SOURCE:
            CHECKPOINT_SOURCE = "downloaded"
        # Update to use persistent storage
        if os.path.exists(ORIGINAL_MODEL_DIR):
            BASE_INIT = ORIGINAL_MODEL_DIR

# =============================================================================
# SUMMARY
# =============================================================================

print("\n" + "=" * 60)
print("📋 CHECKPOINT RESOLUTION SUMMARY")
print("=" * 60)

if RESUME_FROM_CHECKPOINT:
    ckpt_info = get_checkpoint_info(RESUME_FROM_CHECKPOINT)
    print(f"""
   🔄 Mode: RESUME FROM CHECKPOINT
   
   📁 Checkpoint: {ckpt_info['basename']}
   📊 Size: {ckpt_info['size_mb']:.0f} MB
   📅 Date: {ckpt_info['date']}
   📂 Path: {RESUME_FROM_CHECKPOINT}
   
   📦 Base files: {BASE_INIT}
   
   💡 Training will CONTINUE from this checkpoint
      (using lower learning rate for fine-tuning)
""")
else:
    print(f"""
   🆕 Mode: FRESH START
   
   📦 Base model: {BASE_INIT}
   📂 Source: {CHECKPOINT_SOURCE or 'base_model'}
   
   💡 Training will START FROM SCRATCH
      (using standard learning rate)
""")

print("=" * 60)

# Export for use in other cells
print(f"\n🔧 Variables exported:")
print(f"   RESUME_FROM_CHECKPOINT = {'set' if RESUME_FROM_CHECKPOINT else 'None'}")
print(f"   BASE_INIT = {BASE_INIT}")
print(f"   FORCE_FRESH_START = {FORCE_FRESH_START}")

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

LICENSE.txt: 0.00B [00:00, ?B/s]

nam-calm.wav:   0%|          | 0.00/744k [00:00<?, ?B/s]

nam-nhanh.wav:   0%|          | 0.00/646k [00:00<?, ?B/s]

...

nam-truyen-cam.wav:   0%|          | 0.00/876k [00:00<?, ?B/s]

nu-luu-loat.wav:   0%|          | 0.00/711k [00:00<?, ?B/s]

nu-nhe-nhang.wav:   0%|          | 0.00/793k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

✅ Model downloaded: /root/.cache/huggingface/hub/models--capleaf--viXTTS/snapshots/c06f4378883110615941aab481532a9802440b05


In [10]:
# Check GPU
!nvidia-smi

import torch

if torch.cuda.is_available():
    gpu = torch.cuda.get_device_name(0)
    mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"\n✅ GPU: {gpu} ({mem:.1f} GB)")
    if mem < 14:
        print("⚠️  Less than 15GB VRAM - reduce batch size if OOM")
else:
    raise RuntimeError("❌ No GPU found!")

Sun Dec 14 16:02:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:
# Check Storage
import shutil

def check_storage():
    total, used, free = shutil.disk_usage("/")
    free_gb = free // (2**30)
    print(f"💾 Storage: {free_gb}GB free")

    if free_gb < 25:
        print("⚠️  Less than 25GB free - consider reducing dataset size")
    else:
        print("✅ Sufficient storage")

    return free_gb

free_gb = check_storage()

💾 Storage: 63GB free
✅ Sufficient storage


In [12]:
# # Verify transformers and TTS can load GPT2 models
print("🔍 Verifying TTS and transformers compatibility...")

try:
    import torch
    import torchaudio
    print(f"✅ torch version: {torch.__version__}")
    print(f"✅ torchaudio version: {torchaudio.__version__}")
    print(f"✅ CUDA available: {torch.cuda.is_available()}")

    # import torchcodec
    # print("✅ torchcodec version:", torchcodec.__version__)

    import transformers
    print(f"✅ transformers version: {transformers.__version__}")

    from transformers.models.gpt2.modeling_gpt2 import GPT2Model
    print("✅ GPT2Model imports successfully")

    import TTS
    print(f"✅ TTS version: {TTS.__version__}")

    from TTS.tts.configs.xtts_config import XttsConfig
    print("✅ XTTS config imports successfully")

    print("\n🎉 All good: Coqui TTS + transformers + GPT2 are compatible with Python 3.12")

except Exception as e:
    print("❌ CRITICAL: Dependency compatibility issue")
    print(f"   Error: {e}")
    import traceback
    traceback.print_exc()
    raise


🔍 Verifying TTS and transformers compatibility...
✅ torch version: 2.4.1+cu124
✅ torchaudio version: 2.4.1+cu124
✅ CUDA available: True
✅ transformers version: 4.55.4
✅ GPT2Model imports successfully


/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


✅ TTS version: 0.27.2
✅ XTTS config imports successfully

🎉 All good: Coqui TTS + transformers + GPT2 are compatible with Python 3.12


# SECTION 2: Configuration

- Dataset Configuration
- Training Configuration
- Pre-Section 3: Memory Cleanup


In [13]:
# Dataset Configuration
import random

random.seed(42)

# Dataset settings
BASE_SR = 24000        # XTTS-v2 uses 24kHz

# MAX_TRAIN = 4000       # Training samples
# MAX_VAL = 200          # Validation samples
MAX_TRAIN = 2000       # Training samples
MAX_VAL = 100          # Validation samples

# Quality filters
MIN_DURATION = 0.5     # seconds
MAX_DURATION = 11.0    # seconds
MIN_TEXT_LEN = 5       # characters
MAX_TEXT_LEN = 500     # characters

# RAM check each N samples to prevent app crash (ADJUSTED FOR 12GB RAM)
CLEANUP_INTERVAL = 50  # Force garbage collection every 50 samples (reduced from 100)
# CHUNK_SIZE = 500       # Process N samples at a time, also trigger RAM check
CHUNK_SIZE = 300       # Process N samples at a time, also trigger RAM check
RAM_PERCENT_THRESHOLD = 75  # RAM percent threshold (reduced from 85% for safety)
RAM_NUM_THRESHOLD = 6  # GB of RAM threshold (reduced from 8GB for 12GB system)

USE_FLAC = True        # Use compression (saves ~40% space)

print(f"📊 Dataset Config:")
print(f"   Samples: {MAX_TRAIN} train, {MAX_VAL} val")
print(f"   Filters: {MIN_DURATION}-{MAX_DURATION}s, {MIN_TEXT_LEN}-{MAX_TEXT_LEN} chars")
print(f"   Format: {'FLAC' if USE_FLAC else 'WAV'}")
print(f"   RAM Safety: Cleanup every {CLEANUP_INTERVAL} samples, threshold {RAM_PERCENT_THRESHOLD}%")

📊 Dataset Config:
   Samples: 2000 train, 100 val
   Filters: 0.5-11.0s, 5-500 chars
   Format: FLAC
   RAM Safety: Cleanup every 50 samples, threshold 75%


In [14]:
# Training Configuration
import json
import shutil
from datetime import datetime

CFG_PATH = f"{RUN_DIR}/xtts_vi_config.json"

# CRITICAL: Load config from base model instead of creating from scratch
# XTTS requires loading the existing config.json from the checkpoint
BASE_CONFIG = f"{BASE_INIT}/config.json"

print(f"📖 Loading base XTTS config from: {BASE_CONFIG}")

if not os.path.exists(BASE_CONFIG):
    raise FileNotFoundError(f"❌ Base config not found: {BASE_CONFIG}")

# Load the base XTTS config
with open(BASE_CONFIG, 'r', encoding='utf-8') as f:
    config = json.load(f)

print(f"✅ Base config loaded (model: {config.get('model_args', {}).get('xtts', 'unknown')})")

# === DETERMINE RESTORE PATH ===
# Use fine-tuned checkpoint if available, otherwise use base model
if RESUME_FROM_CHECKPOINT and os.path.exists(RESUME_FROM_CHECKPOINT):
    RESTORE_PATH = RESUME_FROM_CHECKPOINT
    IS_RESUMING = True
    # Use lower learning rate when resuming (model already partially trained)
    TRAINING_LR = 1e-5
    TRAINING_EPOCHS = 500  # Fewer epochs needed when resuming
    print(f"\n🔄 RESUMING from checkpoint: {os.path.basename(RESUME_FROM_CHECKPOINT)}")
    print(f"   Using lower LR: {TRAINING_LR} (model already trained)")
else:
    RESTORE_PATH = BASE_INIT
    IS_RESUMING = False
    TRAINING_LR = 5e-5
    TRAINING_EPOCHS = 1000
    print(f"\n🆕 STARTING FRESH from base model")

# Generate unique run name with timestamp
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M")
RUN_NAME = f"vixtts_{'resume' if IS_RESUMING else 'finetune'}_{RUN_TIMESTAMP}"

# Update with fine-tuning specific settings
config.update({
    "output_path": RUN_DIR,
    "run_name": RUN_NAME,

    # Checkpoints - save more frequently to avoid losing progress
    "save_checkpoints": True,
    "save_on_interrupt": True,
    "save_step": 2000,       # Save every 2000 steps (more frequent)
    "save_n_checkpoints": 3, # Keep 3 checkpoints

    # Evaluation - more frequent for better monitoring
    "eval_step": 1000,       # Eval every 1000 steps
    "print_step": 50,        # Print every 50 steps (better visibility)
    "run_eval": True,

    # Model restoration - use resolved path
    "restore_path": RESTORE_PATH,

    # Training (MEMORY-OPTIMIZED for 12GB RAM + T4 15GB VRAM)
    "batch_size": 2,         # Increased from 1
    "grad_accum_steps": 16,  # Effective batch = 32
    "lr": TRAINING_LR,       # Dynamic based on resume status
    "lr_scheduler": "CosineAnnealingLR",
    "lr_scheduler_params": {
        "warmup_steps": 500 if IS_RESUMING else 2000,  # Less warmup when resuming
    },
    "epochs": TRAINING_EPOCHS,
    "mixed_precision": True,

    "datasets": [{
        "formatter": "ljspeech",
        "meta_file_train": f"{MAN_DIR}/train.txt",
        "meta_file_val": f"{MAN_DIR}/val.txt",
        "path": "",
        "language": "vi"
    }],

    "num_loader_workers": 2,  # Increased from 0 for better data loading
    "num_eval_loader_workers": 1,

    # Audio settings (ensure compatibility with our processed data)
    "audio": config.get("audio", {}),

    # Test sentences
    "test_sentences": [
        {"text": "Xin chào! Đây là mô hình tổng hợp giọng nói tiếng Việt.", "language": "vi"},
        {"text": "Chúc bạn một ngày làm việc hiệu quả.", "language": "vi"}
    ]
})

# Save the updated config
with open(CFG_PATH, "w", encoding="utf-8") as f:
    json.dump(config, f, ensure_ascii=False, indent=2)

print(f"\n{'='*60}")
print(f"✅ Training Configuration Saved")
print(f"{'='*60}")
print(f"   📁 Config: {CFG_PATH}")
print(f"   🏷️  Run name: {RUN_NAME}")
print(f"   🔄 Mode: {'RESUME' if IS_RESUMING else 'FRESH START'}")
print(f"   📊 Batch size: {config['batch_size']} × {config['grad_accum_steps']} = {config['batch_size']*config['grad_accum_steps']}")
print(f"   📈 Learning rate: {config['lr']}")
print(f"   🔢 Epochs: {config['epochs']}")
print(f"   💾 Save every: {config['save_step']} steps")
print(f"   📍 Restore from: {os.path.basename(str(RESTORE_PATH))}")


📖 Loading base XTTS config from: /root/.cache/huggingface/hub/models--capleaf--viXTTS/snapshots/c06f4378883110615941aab481532a9802440b05/config.json
✅ Base config loaded (model: unknown)
✅ Fine-tuning config saved: /content/drive/MyDrive/vixtts_runs/vietspeech_run1/xtts_vi_config.json
   Batch size: 1 × 12 = 12
   Workers: 0
   Restore from: /root/.cache/huggingface/hub/models--capleaf--viXTTS/snapshots/c06f4378883110615941aab481532a9802440b05


In [15]:
# Pre-Section 3: Memory Cleanup
import gc
import torch

print("🧹 Cleaning up memory before dataset loading...")

# Force garbage collection
gc.collect()

# Clear any CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Check RAM status
import psutil
mem = psutil.virtual_memory()
free_gb = mem.available // (2**30)
percent_used = mem.percent

print(f"💾 RAM Status:")
print(f"   Available: {free_gb}GB")
print(f"   Used: {percent_used:.1f}%")

if free_gb < 4:
    print("⚠️  WARNING: Low RAM! Dataset loading may fail.")
    print("   Consider reducing MAX_TRAIN/MAX_VAL in Cell 13")
elif free_gb < 6:
    print("⚠️  Moderate RAM available. Monitor closely.")
else:
    print("✅ Sufficient RAM available")

print("\n✅ Ready for dataset preparation")


🧹 Cleaning up memory before dataset loading...
💾 RAM Status:
   Available: 9GB
   Used: 28.7%
✅ Sufficient RAM available

✅ Ready for dataset preparation


# SECTION 3: Dataset Preparation

- Load Dataset
- Process and Save Audio - Setup
- Create Helper Functions (Memory-Optimized)
- Pre-Flight RAM and Storage Check
- Process Samples - Memory-Optimized Main Loop
- Verify Dataset
- Converting JSONL manifests to TSV format for TTS compatibility


In [16]:
# Load Dataset
import sys
import gc
import signal
from contextlib import contextmanager

# CRITICAL: Block TensorFlow/JAX from loading to prevent memory crash
print("🛡️  Blocking TensorFlow/JAX imports to prevent memory exhaustion...")

# Create dummy modules to prevent TensorFlow/JAX from loading
class DummyModule:
    def __getattr__(self, name):
        raise ImportError(f"TensorFlow/JAX blocked to prevent memory crash. Use PyTorch instead.")

# Block TensorFlow and JAX
for module_name in ['tensorflow', 'tf', 'jax', 'jaxlib']:
    if module_name not in sys.modules:
        sys.modules[module_name] = DummyModule()

print("✅ TensorFlow/JAX imports blocked")

# Timeout context manager for hanging operations
@contextmanager
def timeout(seconds):
    def timeout_handler(signum, frame):
        raise TimeoutError(f"Operation timed out after {seconds} seconds")

    # Set the signal handler
    old_handler = signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)
        signal.signal(signal.SIGALRM, old_handler)

# Aggressive memory cleanup before loading
print("🧹 Pre-loading memory cleanup...")
gc.collect()
if 'torch' in sys.modules:
    import torch
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# Now import datasets
from datasets import load_dataset, Features, Value, Audio
import torch, torchaudio, soundfile as sf
import io

# CRITICAL: Monkey-patch the Audio feature to disable automatic decoding
print("🛡️  Patching datasets Audio feature to disable automatic decoding...")
_original_decode = Audio.decode_example

def _no_decode(self, value, token_per_repo_id=None):
    """Return raw audio data without decoding through torchcodec"""
    # Just return the raw value without automatic decoding
    # The value will contain 'path' or 'bytes' that we'll decode manually
    return value

Audio.decode_example = _no_decode
print("✅ Audio auto-decoding disabled - will decode manually")

print("Loading VietSpeech (streaming mode, manual decoding)...")

try:
    # Load dataset - audio will NOT be automatically decoded
    ds = load_dataset(
        "NhutP/VietSpeech",
        split="train",
        streaming=True,
        token=os.environ["HF_TOKEN"]
    )
    print(f"✅ Dataset loaded (streaming iterator created)")

except Exception as e:
    print(f"❌ Failed to load dataset: {e}")
    print(f"   Check your HF_TOKEN and internet connection")
    raise

# Test first sample with manual audio decoding
try:
    print("Testing first sample with manual audio decoding (60s timeout)...")

    with timeout(60):
        first = next(iter(ds))

        # Validate required fields
        if 'audio' not in first or 'transcription' not in first:
            raise ValueError("Dataset missing required fields: 'audio' or 'transcription'")

        # Manually decode audio using soundfile or torchaudio
        audio_data = first['audio']

        # The audio_data now contains raw data (path or bytes) without automatic decoding
        if isinstance(audio_data, dict):
            if audio_data.get('array') is not None:
                # Audio is already decoded (shouldn't happen with our patch, but handle it)
                audio_array = audio_data['array']
                sr = audio_data['sampling_rate']
            elif audio_data.get('bytes') is not None:
                # Audio as bytes - decode with soundfile
                audio_bytes = audio_data['bytes']
                audio_array, sr = sf.read(io.BytesIO(audio_bytes))
            elif audio_data.get('path') is not None:
                # Audio file path - load with soundfile
                audio_path = audio_data['path']
                audio_array, sr = sf.read(audio_path)
            else:
                raise ValueError(f"Audio data format not recognized. Keys: {audio_data.keys()}")
        else:
            raise ValueError(f"Expected audio_data to be dict, got {type(audio_data)}")

        audio_len = len(audio_array) if audio_array is not None else 0
        text_preview = first.get('transcription', '')[:80]

        print(f"✅ First sample verified (manual decoding successful)")
        print(f"   Sample rate: {sr} Hz")
        print(f"   Audio length: {audio_len / sr:.2f}s" if sr > 0 else "   Audio length: unknown")
        print(f"   Text preview: {text_preview}...")

        # Clean up immediately and aggressively
        del first, audio_data, audio_array
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

except TimeoutError as e:
    print(f"❌ CRITICAL: Dataset validation timed out after 60s")
    print(f"   This usually indicates:")
    print(f"   1. Network issues downloading audio data")
    print(f"   2. Dataset backend trying to load large dependencies")
    print(f"   3. Memory allocation failure during audio decoding")
    raise

except Exception as e:
    print(f"❌ CRITICAL: Could not validate first sample: {e}")
    print(f"   Dataset may be corrupted or incompatible")
    import traceback
    traceback.print_exc()
    raise

print("\n✅ Dataset validation passed - Ready to process")

🛡️  Blocking TensorFlow/JAX imports to prevent memory exhaustion...
✅ TensorFlow/JAX imports blocked
🧹 Pre-loading memory cleanup...
🛡️  Patching datasets Audio feature to disable automatic decoding...
✅ Audio auto-decoding disabled - will decode manually
Loading VietSpeech (streaming mode, manual decoding)...


README.md:   0%|          | 0.00/2.30k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

✅ Dataset loaded (streaming iterator created)
Testing first sample with manual audio decoding (60s timeout)...
✅ First sample verified (manual decoding successful)
   Sample rate: 16000 Hz
   Audio length: 4.99s
   Text preview: không mở được...

✅ Dataset validation passed - Ready to process


In [17]:
# Process and Save Audio - Setup
import json, time
import gc
import psutil

# Force aggressive memory cleanup before starting
print("🧹 Aggressive memory cleanup before processing...")
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Check RAM before starting
mem = psutil.virtual_memory()
free_gb = mem.available // (2**30)
if free_gb < 6:
    print(f"⚠️  WARNING: Only {free_gb}GB RAM available!")
    print(f"   Consider reducing MAX_TRAIN/MAX_VAL in Cell 13")

WAV_DIR = f"{DATA_DIR}/wavs_vietspeech"
os.makedirs(WAV_DIR, exist_ok=True)
os.makedirs(MAN_DIR, exist_ok=True)

# Counter variables (no large lists in memory)
train_count, val_count = 0, 0
processed, skipped = 0, {"duration": 0, "text": 0, "invalid": 0}
start_time = time.time()

train_manifest = f"{MAN_DIR}/train.jsonl"
val_manifest = f"{MAN_DIR}/val.jsonl"

# Clear existing manifests (for clean reruns)
with open(train_manifest, 'w', encoding='utf-8') as f:
    pass  # Create empty file
with open(val_manifest, 'w', encoding='utf-8') as f:
    pass  # Create empty file

print("="*60)
print(f"PROCESSING DATASET (Incremental Write Mode)")
print(f"Target: {MAX_TRAIN} train + {MAX_VAL} val")
print(f"Available RAM: {free_gb}GB")
print("="*60 + "\n")

🧹 Aggressive memory cleanup before processing...
PROCESSING DATASET (Incremental Write Mode)
Target: 2000 train + 100 val
Available RAM: 8GB



In [18]:
# Helper Functions (Memory-Optimized)
def resample_to_24k(waveform, sr):
    """Resample to 24kHz mono - optimized for memory"""
    if waveform.ndim == 1:
        waveform = waveform.unsqueeze(0)
    if sr != BASE_SR:
        waveform = torchaudio.functional.resample(waveform, sr, BASE_SR)
    if waveform.size(0) > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    return waveform.squeeze(0)

def monitor_storage():
    """Monitor free storage"""
    total, used, free = shutil.disk_usage("/")
    return free // (2**30)

def monitor_ram():
    """Monitor RAM usage - returns (free_gb, percent_used)"""
    mem = psutil.virtual_memory()
    free_gb = mem.available // (2**30)
    percent = mem.percent
    return free_gb, percent

def aggressive_cleanup():
    """Force immediate memory cleanup"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def write_sample_immediate(item, train_count, val_count, train_file, val_file, target_train, target_val):
    """Write sample to manifest immediately WITHOUT storing in memory list"""
    if val_count < target_val:
        with open(val_file, 'a', encoding='utf-8') as f:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")
        return 'val', train_count, val_count + 1
    elif train_count < target_train:
        with open(train_file, 'a', encoding='utf-8') as f:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")
        return 'train', train_count + 1, val_count
    return None, train_count, val_count

def check_ram_critical():
    """Check if RAM is critically low - returns True if should stop"""
    free_gb, ram_percent = monitor_ram()

    if ram_percent > 95:
        print(f"\n🚨 CRITICAL: RAM at {ram_percent}%! Stopping immediately!")
        return True
    elif ram_percent > 90:
        print(f"\n⚠️  WARNING: RAM at {ram_percent}%, only {free_gb}GB free!")
        aggressive_cleanup()
        return False

    return False

print("✅ Helper functions loaded (memory-optimized)")

✅ Helper functions loaded (memory-optimized)


In [19]:
# Pre-Flight RAM and Storage Check
print("="*60)
print("PRE-PROCESSING SYSTEM CHECK")
print("="*60)

# Check RAM
free_gb, ram_percent = monitor_ram()
print(f"💾 RAM Status:")
print(f"   Available: {free_gb}GB")
print(f"   Used: {ram_percent:.1f}%")

# Conservative estimate (no longer keeping all samples in memory)
estimated_peak_gb = 4  # Peak during processing with aggressive cleanup

if free_gb < estimated_peak_gb:
    print(f"\n⚠️  WARNING: Low RAM ({free_gb}GB available, recommend {estimated_peak_gb}GB+)")
    print(f"   Processing will continue with extra caution")
    print(f"   Consider reducing MAX_TRAIN/MAX_VAL if issues occur")
else:
    print(f"✅ Sufficient RAM for processing")

# Check storage
free_storage = monitor_storage()
estimated_storage = ((MAX_TRAIN + MAX_VAL) * 0.5) // 1024  # ~500KB per FLAC file
print(f"\n💿 Storage Status:")
print(f"   Available: {free_storage}GB")
print(f"   Estimated need: ~{estimated_storage}GB")

if free_storage < estimated_storage + 10:
    print(f"⚠️  WARNING: Low storage! Consider reducing dataset size")
else:
    print(f"✅ Sufficient storage")

print("\n✅ System check complete - Starting dataset processing...")
print("="*60 + "\n")

PRE-PROCESSING SYSTEM CHECK
💾 RAM Status:
   Available: 8GB
   Used: 30.6%
✅ Sufficient RAM for processing

💿 Storage Status:
   Available: 63GB
   Estimated need: ~1.0GB
✅ Sufficient storage

✅ System check complete - Starting dataset processing...



In [20]:
# Process Samples - Memory-Optimized Main Loop
# Safety limit: prevent infinite loops
MAX_ITERATIONS = (MAX_TRAIN + MAX_VAL) * 10  # 10x buffer for filtering
iteration_count = 0

# Dynamic targets (can be adjusted for low RAM)
target_train = MAX_TRAIN
target_val = MAX_VAL

try:
    for i, row in enumerate(ds):
        iteration_count += 1

        # CRITICAL: Check RAM before processing each sample
        if check_ram_critical():
            print(f"  Emergency stop at {processed} samples to prevent crash")
            break

        # Safety: prevent infinite loops
        if iteration_count > MAX_ITERATIONS:
            print(f"\n⚠️  Reached maximum iterations ({MAX_ITERATIONS})")
            print(f"  Processed: {processed} samples")
            break

        # Check if target reached
        if train_count >= target_train and val_count >= target_val:
            print(f"\n✅ Target reached!")
            break

        # Extract data
        try:
            au = row["audio"]
            text = (row.get("transcription") or "").strip()
        except Exception as e:
            skipped["invalid"] += 1
            continue

        # Validate text
        if not text or au is None:
            skipped["invalid"] += 1
            continue

        if len(text) < MIN_TEXT_LEN or len(text) > MAX_TEXT_LEN:
            skipped["text"] += 1
            continue

        # Process audio with manual decoding (fallback from torchcodec)
        try:
            # Manually decode audio based on available format
            if au.get("array") is not None:
                # Audio is already decoded
                audio_array = au["array"]
                sr = int(au["sampling_rate"])
            elif au.get("bytes") is not None:
                # Audio needs manual decoding from bytes
                import io
                audio_bytes = au["bytes"]
                audio_array, sr = sf.read(io.BytesIO(audio_bytes))
            elif au.get("path") is not None:
                # Audio needs loading from path
                audio_array, sr = sf.read(au["path"])
            else:
                skipped["invalid"] += 1
                continue

            # Convert to torch tensor
            if isinstance(audio_array, torch.Tensor):
                wav = audio_array.float()
            else:
                wav = torch.tensor(audio_array, dtype=torch.float32)

            # Flatten to 1D if needed
            if wav.ndim > 1:
                wav = wav.mean(dim=-1)

            duration = len(wav) / sr

            if duration < MIN_DURATION or duration > MAX_DURATION:
                skipped["duration"] += 1
                del wav  # Immediate cleanup
                continue

            # Resample and save
            wav24 = resample_to_24k(wav, sr)
            ext = ".flac" if USE_FLAC else ".wav"
            audio_path = f"{WAV_DIR}/vs_{processed:08d}{ext}"

            sf.write(audio_path, wav24.numpy(), BASE_SR)

            # CRITICAL: Delete tensors immediately after use
            del wav, wav24

        except Exception as e:
            print(f"\n⚠️  Error processing audio: {e}")
            skipped["invalid"] += 1
            continue

        # Create manifest entry
        item = {
            "audio_file": audio_path,
            "text": text,
            "speaker_name": "vietspeech",
            "language": "vi"
        }

        # Write immediately to disk (NO memory accumulation)
        split, train_count, val_count = write_sample_immediate(
            item, train_count, val_count,
            train_manifest, val_manifest,
            target_train, target_val
        )

        if split:
            processed += 1
        else:
            # Should not reach here if checks above work
            break

        # AGGRESSIVE: Cleanup every CLEANUP_INTERVAL samples
        if processed % CLEANUP_INTERVAL == 0:
            aggressive_cleanup()

        # Progress report every 500 samples
        if processed % 500 == 0:
            free_gb, ram_percent = monitor_ram()
            elapsed = time.time() - start_time
            rate = processed / elapsed if elapsed > 0 else 0

            print(f"Processed: {processed:5d} | Train: {train_count:5d} | Val: {val_count:3d} | "
                  f"Rate: {rate:.1f}/s | RAM: {ram_percent:.0f}% | Storage: {monitor_storage()}GB")

            # Dynamic adjustment for low RAM (using configured thresholds)
            if free_gb < RAM_NUM_THRESHOLD or ram_percent > RAM_PERCENT_THRESHOLD:
                print(f"⚠️  RAM threshold reached ({free_gb}GB, {ram_percent:.0f}%) - reducing target size")
                target_train = min(target_train, processed + 2000)
                target_val = min(target_val, val_count + 100)

except KeyboardInterrupt:
    print(f"\n⚠️  Interrupted by user at {processed} samples")
except Exception as e:
    print(f"\n❌ Fatal error during processing: {e}")
    print(f"   Processed {processed} samples before failure")
    raise
finally:
    # Final cleanup
    aggressive_cleanup()

# Summary
elapsed = time.time() - start_time
total = processed + sum(skipped.values())

print("\n" + "="*60)
print("✅ DATASET PREPARATION COMPLETE")
print("="*60)
print(f"📊 Examined: {total}, Accepted: {processed} ({processed/total*100:.1f}%)" if total > 0 else "📊 No samples processed")
print(f"   Skipped: {sum(skipped.values())} (invalid:{skipped['invalid']}, duration:{skipped['duration']}, text:{skipped['text']})")
print(f"\n📁 Output:")
print(f"   Train: {train_count} samples → {train_manifest}")
print(f"   Val:   {val_count} samples → {val_manifest}")
print(f"\n⏱️  Time: {elapsed/60:.1f} min ({processed/elapsed:.1f} samples/s)" if elapsed > 0 else "⏱️  Time: <1s")
free_gb, ram_percent = monitor_ram()
print(f"💾 RAM: {ram_percent:.0f}% used ({free_gb}GB free) | Storage: {monitor_storage()}GB free")
print("="*60)

Processed:   500 | Train:   400 | Val: 100 | Rate: 42.1/s | RAM: 37% | Storage: 63GB
Processed:  1000 | Train:   900 | Val: 100 | Rate: 44.8/s | RAM: 43% | Storage: 63GB
Processed:  1500 | Train:  1400 | Val: 100 | Rate: 45.0/s | RAM: 48% | Storage: 63GB
Processed:  2000 | Train:  1900 | Val: 100 | Rate: 45.6/s | RAM: 51% | Storage: 63GB

✅ Target reached!

✅ DATASET PREPARATION COMPLETE
📊 Examined: 2100, Accepted: 2100 (100.0%)
   Skipped: 0 (invalid:0, duration:0, text:0)

📁 Output:
   Train: 2000 samples → /content/datasets/manifests_vietspeech/train.jsonl
   Val:   100 samples → /content/datasets/manifests_vietspeech/val.jsonl

⏱️  Time: 0.8 min (45.6 samples/s)
💾 RAM: 51% used (6GB free) | Storage: 63GB free


In [21]:
# Verify Dataset
print("🔍 Verifying generated dataset...")

# Read first line from train manifest
try:
    with open(train_manifest, 'r', encoding='utf-8') as f:
        first_line = f.readline().strip()
        if not first_line:
            raise ValueError("Train manifest is empty!")
        test = json.loads(first_line)

    # Verify audio file exists and is readable
    if not os.path.exists(test["audio_file"]):
        raise FileNotFoundError(f"Audio file not found: {test['audio_file']}")

    audio, sr = sf.read(test["audio_file"])

    print("✅ Dataset verification:")
    print(f"   Sample rate: {sr} Hz (expected: {BASE_SR})")
    print(f"   Duration: {len(audio)/sr:.2f}s")
    print(f"   Text: {test['text'][:100]}...")
    print(f"   Train samples: {train_count}")
    print(f"   Val samples: {val_count}")

    assert sr == BASE_SR, f"Sample rate mismatch: {sr} != {BASE_SR}"
    print("\n✅ All checks passed!")

except Exception as e:
    print(f"❌ Verification failed: {e}")
    print(f"   Check that dataset processing completed successfully")
    raise

🔍 Verifying generated dataset...
✅ Dataset verification:
   Sample rate: 24000 Hz (expected: 24000)
   Duration: 4.74s
   Text: sau đó ngân hàng sẽ thanh toán bảy mươi phần trăm còn lại cho chủ đầu tư...
   Train samples: 2000
   Val samples: 100

✅ All checks passed!


In [22]:
# Converting JSONL manifests to TSV format for TTS compatibility
import json
import os

print("🔧 Converting JSONL manifests to TSV format for TTS compatibility...")

# Read JSONL manifests
def convert_jsonl_to_tsv(jsonl_path, tsv_path):
    """Convert JSONL manifest to TSV format (pipe-separated with header)"""
    if not os.path.exists(jsonl_path):
        raise FileNotFoundError(f"JSONL file not found: {jsonl_path}\nPlease run the dataset preparation cells first!")

    with open(jsonl_path, 'r', encoding='utf-8') as f_in:
        with open(tsv_path, 'w', encoding='utf-8') as f_out:
            # Write header - CRITICAL for coqui formatter
            f_out.write("audio_file|text|speaker_name\n")

            for line in f_in:
                item = json.loads(line.strip())
                # Format: audio_path|text|speaker_name
                tsv_line = f"{item['audio_file']}|{item['text']}|{item.get('speaker_name', 'vietspeech')}\n"
                f_out.write(tsv_line)

# Convert both train and val manifests
train_jsonl = f"{MAN_DIR}/train.jsonl"
val_jsonl = f"{MAN_DIR}/val.jsonl"
train_tsv = f"{MAN_DIR}/train.txt"
val_tsv = f"{MAN_DIR}/val.txt"

# Check if JSONL files exist
if not os.path.exists(train_jsonl) or not os.path.exists(val_jsonl):
    print("❌ JSONL manifest files not found!")
    print(f"   Expected: {train_jsonl}")
    print(f"   Expected: {val_jsonl}")
    print()
    print("⚠️  Please run the dataset preparation cells (Section 3) first:")
    print("   1. Cell: Load Dataset")
    print("   2. Cell: Process and Save Audio - Setup")
    print("   3. Cell: Helper Functions")
    print("   4. Cell: Process Samples - Memory-Optimized Main Loop")
    raise FileNotFoundError("Dataset preparation not complete. Run dataset cells first!")

# ALWAYS regenerate TSV files (don't skip) to ensure proper header
print(f"📄 Converting JSONL to TSV with header row...")

# Delete old TSV files if they exist
for tsv_file in [train_tsv, val_tsv]:
    if os.path.exists(tsv_file):
        os.remove(tsv_file)
        print(f"   Deleted old: {tsv_file}")

# Convert with header
convert_jsonl_to_tsv(train_jsonl, train_tsv)
convert_jsonl_to_tsv(val_jsonl, val_tsv)

print(f"✅ Converted manifests to TSV format with headers:")
print(f"   Train: {train_tsv}")
print(f"   Val: {val_tsv}")

# Verify format
with open(train_tsv, 'r', encoding='utf-8') as f:
    header = f.readline().strip()
    first_line = f.readline().strip()
    parts = first_line.split('|')
    print(f"\n📝 TSV format verified:")
    print(f"   Header: {header}")
    print(f"   Sample: audio={parts[0][:40]}... | text={parts[1][:30]}... | speaker={parts[2]}")


🔧 Converting JSONL manifests to TSV format for TTS compatibility...
📄 Converting JSONL to TSV with header row...
✅ Converted manifests to TSV format with headers:
   Train: /content/datasets/manifests_vietspeech/train.txt
   Val: /content/datasets/manifests_vietspeech/val.txt

📝 TSV format verified:
   Header: audio_file|text|speaker_name
   Sample: audio=/content/datasets/wavs_vietspeech/vs_000... | text=sau đó ngân hàng sẽ thanh toán... | speaker=vietspeech


# SECTION 4: Training

- Check XTTS Training Method
- Pre-Training Memory Cleanup
- Pre-Training Checks
- Start Training
- Monitor Training (Optional)


In [23]:
# Check XTTS Training Method
import os
import glob
import sys

print("🔍 Checking XTTS training options...\n")

# Dynamically find TTS bin path based on Python version
python_version = f"{sys.version_info.major}.{sys.version_info.minor}"
tts_bin_path = f"/usr/local/lib/python{python_version}/dist-packages/TTS/bin"

print(f"Python version: {python_version}")
print(f"Looking for TTS bin at: {tts_bin_path}")

# Check what training scripts are available
if os.path.exists(tts_bin_path):
    train_scripts = glob.glob(f"{tts_bin_path}/train*.py")
    print("\nAvailable training scripts:")
    for script in train_scripts:
        print(f"   - {os.path.basename(script)}")
else:
    print("⚠️  Could not find TTS bin directory")
    print(f"   Searched: {tts_bin_path}")

# Check if XTTS trainer exists
xtts_trainer = f"{tts_bin_path}/train_gpt_xtts.py"
if os.path.exists(xtts_trainer):
    print("\n✅ XTTS-specific trainer found: train_gpt_xtts.py")
    print("   Will use this for better XTTS fine-tuning")
    USE_XTTS_TRAINER = True
else:
    print("\n⚠️  No XTTS-specific trainer, using standard train_tts.py")
    USE_XTTS_TRAINER = False

print("\n✅ Training method determined")


🔍 Checking XTTS training options...

Python version: 3.12
Looking for TTS bin at: /usr/local/lib/python3.12/dist-packages/TTS/bin

Available training scripts:
   - train_vocoder.py
   - train_encoder.py
   - train_tts.py

⚠️  No XTTS-specific trainer, using standard train_tts.py

✅ Training method determined


In [24]:
# Pre-Training Memory Cleanup
# Free up as much RAM as possible before training starts

import gc
import torch
import psutil

print("🧹 CRITICAL: Pre-training aggressive memory cleanup...")
print("=" * 60)

# Check RAM before cleanup
mem_before = psutil.virtual_memory()
print(f"📊 RAM Before Cleanup: {mem_before.percent:.0f}% used ({mem_before.available // (2**30)}GB free)")

# Delete any large objects from dataset preparation
try:
    if 'ds' in globals():
        del ds
        print("✅ Deleted dataset iterator")
except:
    pass

try:
    if 'first' in globals():
        del first
        print("✅ Deleted test samples")
except:
    pass

# Force aggressive garbage collection (multiple passes)
for i in range(3):
    collected = gc.collect()
    print(f"✅ GC pass {i+1}: Collected {collected} objects")

# Clear CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    print("✅ Cleared CUDA cache")

# Check RAM after cleanup
mem_after = psutil.virtual_memory()
freed_gb = (mem_after.available - mem_before.available) // (2**30)

print(f"\n📊 RAM After Cleanup: {mem_after.percent:.0f}% used ({mem_after.available // (2**30)}GB free)")
print(f"💾 Freed: ~{abs(freed_gb)}GB")
print("="*60)

if mem_after.available < 7 * (2**30):  # Less than 7GB free
    print("⚠️  WARNING: Low RAM! Training may crash.")
    print("   Consider:")
    print("   1. Restart runtime and skip dataset preparation")
    print("   2. Further reduce batch_size to 1")
    print("   3. Reduce dataset size in Cell 14")
else:
    print("✅ Sufficient RAM available for training")


🧹 CRITICAL: Pre-training aggressive memory cleanup...
📊 RAM Before Cleanup: 51% used (6GB free)
✅ Deleted dataset iterator
✅ GC pass 1: Collected 8 objects
✅ GC pass 2: Collected 0 objects
✅ GC pass 3: Collected 0 objects
✅ Cleared CUDA cache

📊 RAM After Cleanup: 52% used (6GB free)
💾 Freed: ~1GB
⚠️  WARNING: Low RAM! Training may crash.
   Consider:
   1. Restart runtime and skip dataset preparation
   2. Further reduce batch_size to 1
   3. Reduce dataset size in Cell 14


In [25]:
# Pre-Training Checks
import json

print("="*60)
print("PRE-TRAINING CHECKLIST")
print("="*60)

# Verify config is valid JSON and has required fields
config_valid = False
if os.path.exists(CFG_PATH):
    try:
        with open(CFG_PATH, 'r') as f:
            cfg = json.load(f)
        # Check for essential XTTS fields
        has_model_args = 'model_args' in cfg or 'model' in cfg
        has_restore = 'restore_path' in cfg
        config_valid = has_model_args and has_restore
        if config_valid:
            print(f"📋 Config validation:")
            print(f"   ✅ Has model architecture: {has_model_args}")
            print(f"   ✅ Has restore path: {has_restore}")
            print(f"   ✅ Restore from: {cfg.get('restore_path', 'N/A')}")
        else:
            print(f"⚠️  Config missing required fields:")
            print(f"   Model args: {has_model_args}, Restore: {has_restore}")
    except Exception as e:
        print(f"❌ Config validation failed: {e}")

checks = {
    "GPU": torch.cuda.is_available(),
    "Config exists": os.path.exists(CFG_PATH),
    "Config valid": config_valid,
    "Manifests": os.path.exists(train_manifest) and os.path.exists(val_manifest),
    "Storage": monitor_storage() >= 20
}

print(f"\n{'='*60}")
for name, passed in checks.items():
    status = "✅" if passed else "❌"
    print(f"{status} {name}")

if not all(checks.values()):
    print(f"\n❌ Some checks failed!")
    if not config_valid:
        print(f"   Config issue: Try re-running Cell 15 (Training Configuration)")
    raise RuntimeError("❌ Pre-training checks failed")

print("\n✅ All checks passed! Ready to train.")

PRE-TRAINING CHECKLIST
📋 Config validation:
   ✅ Has model architecture: True
   ✅ Has restore path: True
   ✅ Restore from: /root/.cache/huggingface/hub/models--capleaf--viXTTS/snapshots/c06f4378883110615941aab481532a9802440b05

✅ GPU
✅ Config exists
✅ Config valid
✅ Manifests
✅ Storage

✅ All checks passed! Ready to train.


In [ ]:
# Start Training - Using XTTS Demo
print("🚀 Starting XTTS fine-tuning...")
print(f"   Config: {CFG_PATH}")
print(f"   Restore from: {BASE_INIT}")
print(f"   Output: {RUN_DIR}\n")

# ============================================================
# CRITICAL: Patch XTTS to support Vietnamese language
# ============================================================
print("🔧 Patching XTTS tokenizer to support Vietnamese ('vi')...")

# Patch the language check in the trainer dataset
from TTS.tts.layers.xtts.trainer import dataset as xtts_dataset
import random

_original_get_text = xtts_dataset.XTTSDataset.get_text

def _patched_get_text(self, text, lang):
    # Map Vietnamese to English for tokenization
    # The viXTTS model handles Vietnamese text internally
    if lang == 'vi':
        lang = 'en'  # Use English tokenizer as fallback
    return _original_get_text(self, text, lang)

xtts_dataset.XTTSDataset.get_text = _patched_get_text
print("   ✅ Patched XTTSDataset.get_text to handle Vietnamese")

# Patch the split_sentence function to handle Vietnamese
import TTS.tts.layers.xtts.tokenizer as tokenizer_module

_original_split = tokenizer_module.split_sentence

def _patched_split_sentence(text, lang, text_split_length=250):
    if lang == 'vi':
        lang = 'en'  # Fallback for sentence splitting
    return _original_split(text, lang, text_split_length)

tokenizer_module.split_sentence = _patched_split_sentence
print("   ✅ Patched split_sentence to handle Vietnamese")

# ============================================================
# FIX: Patch __getitem__ to prevent infinite recursion
# The original code uses `return self[1]` as fallback which causes
# infinite recursion if sample 1 also fails.
#
# Solution: Use an instance flag to BLOCK the original's internal
# retry mechanism (self[1]) from triggering recursion.
# ============================================================
_original_getitem = xtts_dataset.XTTSDataset.__getitem__

def _patched_getitem(self, index):
    """
    Patched __getitem__ that prevents infinite recursion.
    Uses an instance flag to block the original's internal self[1] fallback.
    """
    MAX_RETRIES = 10

    # Check if we're being called from the ORIGINAL's internal self[1] fallback
    # If so, immediately raise to prevent recursion - our outer code handles retry
    if getattr(self, '_xtts_in_getitem', False):
        # This call came from the original's internal error handling (self[1])
        # Raise immediately to break the recursion chain
        raise StopIteration(f"Blocked internal retry for index {index}")

    # Track which indices we've already tried
    tried_indices = set()
    last_error = None
    current_index = index

    for attempt in range(MAX_RETRIES):
        tried_indices.add(current_index)

        try:
            # Set flag BEFORE calling original - this blocks its internal self[1]
            self._xtts_in_getitem = True
            try:
                result = _original_getitem(self, current_index)
                return result
            finally:
                # Always clear the flag after the call
                self._xtts_in_getitem = False

        except StopIteration:
            # This shouldn't happen in normal flow, but handle it
            self._xtts_in_getitem = False
            raise

        except Exception as e:
            self._xtts_in_getitem = False
            last_error = e

            # Pick a new random index we haven't tried yet
            total_samples = len(self)
            available = [i for i in range(total_samples) if i not in tried_indices]

            if not available:
                # We've tried all samples, give up
                break

            current_index = random.choice(available)

    # All retries exhausted
    print(f"\n⚠️ WARNING: Failed to load sample after {len(tried_indices)} attempts.")
    print(f"   Tried indices: {sorted(tried_indices)[:10]}{'...' if len(tried_indices) > 10 else ''}")
    print(f"   Last error: {type(last_error).__name__}: {last_error}")
    print(f"   Skipping problematic batch...")
    raise RuntimeError(f"Dataset error after {len(tried_indices)} retries: {last_error}")

xtts_dataset.XTTSDataset.__getitem__ = _patched_getitem
print("   ✅ Patched XTTSDataset.__getitem__ to prevent infinite recursion")

# ============================================================
# FIX: Patch DataLoader to use fewer workers (prevent OOM)
# The train_gpt function uses 8 workers by default, which causes
# OOM on systems with limited RAM (12GB Colab).
# ============================================================
import torch.utils.data as torch_data

_OriginalDataLoader = torch_data.DataLoader

class _MemoryOptimizedDataLoader(_OriginalDataLoader):
    """DataLoader wrapper that limits num_workers to prevent OOM."""
    def __init__(self, *args, **kwargs):
        # CRITICAL: Force num_workers to 0 in Colab/Jupyter to prevent
        # 'can only test a child process' AssertionError from multiprocessing
        if kwargs.get('num_workers', 0) > 0:
            print(f"   ⚠️ Forcing num_workers from {kwargs['num_workers']} to 0 (multiprocessing fix)")
            kwargs['num_workers'] = 0
        super().__init__(*args, **kwargs)

torch_data.DataLoader = _MemoryOptimizedDataLoader
print("   ✅ Patched DataLoader to limit workers (OOM prevention)")

# ============================================================
# FIX: Patch Trainer to save checkpoints more frequently
# train_gpt doesn't expose save_step parameter, so we patch Trainer
# ============================================================
import trainer.trainer as trainer_module

_OriginalTrainer = trainer_module.Trainer

class _CheckpointFrequentTrainer(_OriginalTrainer):
    """Trainer wrapper that forces frequent checkpoint saving."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Override checkpoint settings after init
        if hasattr(self, 'config'):
            # Save every 500 steps (instead of default which might be very high)
            old_save_step = getattr(self.config, 'save_step', 'unknown')
            self.config.save_step = 500
            self.config.save_n_checkpoints = 3
            self.config.save_checkpoints = True
            self.config.save_best_after = 100  # Start saving best after 100 steps
            print(f"   ⚠️ Modified checkpoint settings: save_step {old_save_step} → 500")

trainer_module.Trainer = _CheckpointFrequentTrainer
print("   ✅ Patched Trainer to save checkpoints every 500 steps")

print("✅ All patches applied!\n")

# ============================================================
# Now proceed with training
# ============================================================
from TTS.demos.xtts_ft_demo.xtts_demo import train_gpt
import json
import inspect

# Load config
print("📖 Loading configuration...")
with open(CFG_PATH, 'r') as f:
    cfg = json.load(f)

# Extract training parameters
train_csv = cfg['datasets'][0]['meta_file_train']
eval_csv = cfg['datasets'][0]['meta_file_val']
output_path = cfg['output_path']
batch_size = cfg.get('batch_size', 4)
grad_acum = cfg.get('grad_accum_steps', 3)
max_epochs = cfg.get('epochs', 1000)

print(f"✅ Configuration loaded")
print(f"   Batch size: {batch_size} × {grad_acum} = {batch_size * grad_acum}")
print(f"   Max epochs: {max_epochs}")
print(f"   Train manifest: {train_csv}")
print(f"   Val manifest: {eval_csv}\n")

print("="*60)
print("🚀 STARTING XTTS GPT TRAINING...")
print("="*60)
print()
print("⚠️  NOTE: Checkpoints will be saved to a SUBDIRECTORY:")
print(f"   {output_path}/run/training/GPT_XTTS_FT-<timestamp>/")
print("   The 'Find Best Checkpoint' cell will search recursively.")
print()

# Start training using official XTTS demo function
train_gpt(
    language="vi",  # Will be handled by our patch
    num_epochs=max_epochs,
    batch_size=batch_size,
    grad_acumm=grad_acum,
    train_csv=train_csv,
    eval_csv=eval_csv,
    output_path=output_path,
)


🚀 Starting XTTS fine-tuning...
   Config: /content/drive/MyDrive/vixtts_runs/vietspeech_run1/xtts_vi_config.json
   Restore from: /root/.cache/huggingface/hub/models--capleaf--viXTTS/snapshots/c06f4378883110615941aab481532a9802440b05
   Output: /content/drive/MyDrive/vixtts_runs/vietspeech_run1

🔧 Patching XTTS tokenizer to support Vietnamese ('vi')...
   ✅ Patched XTTSDataset.get_text to handle Vietnamese
   ✅ Patched split_sentence to handle Vietnamese
   ✅ Patched XTTSDataset.__getitem__ to prevent infinite recursion
   ✅ Patched DataLoader to limit workers (OOM prevention)
   ✅ Patched Trainer to save checkpoints every 500 steps
✅ All patches applied!

📖 Loading configuration...
✅ Configuration loaded
   Batch size: 1 × 12 = 12
   Max epochs: 1000
   Train manifest: /content/datasets/manifests_vietspeech/train.txt
   Val manifest: /content/datasets/manifests_vietspeech/val.txt

🚀 STARTING XTTS GPT TRAINING...

⚠️  NOTE: Checkpoints will be saved to a SUBDIRECTORY:
   /content/drive

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 16
 | > Torch seed: 1
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/content/drive/MyDrive/vixtts_runs/vietspeech_run1/run/training/GPT_XTTS_FT-December-14-2025_04+06PM-0000000

 > Model has 518442047 parameters

 > EPOCH: 0/999
 --> /content/drive/MyDrive/vixtts_runs/vietspeech_run1/run/training/GPT_XTTS_FT-December-14-2025_04+06PM-0000000

 > TRAINING (2025-12-14 16:06:17) 


   ⚠️ Reducing num_workers from 8 to 2 (OOM prevention)



 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 3.299548625946045 (+0.0)
     | > avg_loss_text_ce: 0.06084710359573364 (+0.0)
     | > avg_loss_mel_ce: 5.422955513000488 (+0.0)
     | > avg_loss: 5.483802795410156 (+0.0)


 > EPOCH: 1/999
 --> /content/drive/MyDrive/vixtts_runs/vietspeech_run1/run/training/GPT_XTTS_FT-December-14-2025_04+06PM-0000000

 > TRAINING (2025-12-14 16:06:32) 

 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.02995467185974121 (-3.2695939540863037)
     | > avg_loss_text_ce: 0.06084710359573364 (+0.0)
     | > avg_loss_mel_ce: 5.422955513000488 (+0.0)
     | > avg_loss: 5.483802795410156 (+0.0)


 > EPOCH: 2/999
 --> /content/drive/MyDrive/vixtts_runs/vietspeech_run1/run/training/GPT_XTTS_FT-December-14-2025_04+06PM-0000000

 > TRAINING (2025-12-14 16:06:32) 

 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.02725076675415039 (-0.0027039051055908203)
     | > avg_loss_text_ce: 0.06084710359573364 (+0.0)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7da73b38cc20>Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x7da73b38cc20>Traceback (most recent call last):

  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1477, in __del__
Traceback (most recent call last):
      File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers

      File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
if w.is_alive():
      if w.is_alive(): 
         ^^ ^ ^^^^^^^^^^^^^^^^^^^^^

  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
        assert self.

...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7da73b38cc20>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7da73b38cc20>Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1477, in __del__

Traceback (most recent call last):
    self._shutdown_workers()  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1477, in __del__

    self._shutdown_workers()  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers

      File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
if w.is_alive():    
if w.is_alive(): 
             ^^^^^^^^^^^^^^^^^^^^^^^^

  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
        assert self.


 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.11071991920471191 (+0.06808042526245117)
     | > avg_loss_text_ce: 0.06084710359573364 (+0.0)
     | > avg_loss_mel_ce: 5.422955513000488 (+0.0)
     | > avg_loss: 5.483802795410156 (+0.0)


 > EPOCH: 728/999
 --> /content/drive/MyDrive/vixtts_runs/vietspeech_run1/run/training/GPT_XTTS_FT-December-14-2025_04+06PM-0000000

 > TRAINING (2025-12-14 16:59:21) 

 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.06077933311462402 (-0.04994058609008789)
     | > avg_loss_text_ce: 0.06084710359573364 (+0.0)
     | > avg_loss_mel_ce: 5.422955513000488 (+0.0)
     | > avg_loss: 5.483802795410156 (+0.0)


 > EPOCH: 729/999
 --> /content/drive/MyDrive/vixtts_runs/vietspeech_run1/run/training/GPT_XTTS_FT-December-14-2025_04+06PM-0000000

 > TRAINING (2025-12-14 16:59:21) 

 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.05913519859313965 (-0.001644134521484375)
     | > avg_loss_text_ce: 0.06

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7da73b38cc20>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
    if w.is_alive():
Exception ignored in:   <function _MultiProcessingDataLoaderIter.__del__ at 0x7da73b38cc20>
 Traceback (most recent call last):
    File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1477, in __del__
      ^self._shutdown_workers()
^^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
^    ^if w.is_alive():^
^  ^^ ^ ^   ^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
^    ^assert self._parent_pid == os.getpid(), 'can only test a child process'^
^^    ^ ^  ^ ^
  File "/usr/li


 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0713045597076416 (+0.012169361114501953)
     | > avg_loss_text_ce: 0.06084710359573364 (+0.0)
     | > avg_loss_mel_ce: 5.422955513000488 (+0.0)
     | > avg_loss: 5.483802795410156 (+0.0)


 > EPOCH: 731/999
 --> /content/drive/MyDrive/vixtts_runs/vietspeech_run1/run/training/GPT_XTTS_FT-December-14-2025_04+06PM-0000000

 > TRAINING (2025-12-14 16:59:34) 

 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.04074716567993164 (-0.03055739402770996)
     | > avg_loss_text_ce: 0.06084710359573364 (+0.0)
     | > avg_loss_mel_ce: 5.422955513000488 (+0.0)
     | > avg_loss: 5.483802795410156 (+0.0)


 > EPOCH: 732/999
 --> /content/drive/MyDrive/vixtts_runs/vietspeech_run1/run/training/GPT_XTTS_FT-December-14-2025_04+06PM-0000000

 > TRAINING (2025-12-14 16:59:34) 

 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0476839542388916 (+0.006936788558959961)
     | > avg_loss_text_ce: 0.060

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7da73b38cc20><function _MultiProcessingDataLoaderIter.__del__ at 0x7da73b38cc20>

Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1477, in __del__
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1477, in __del__


In [ ]:
# # Monitor Training (Optional)
# # Load TensorBoard
# %load_ext tensorboard
# %tensorboard --logdir "{RUN_DIR}"

# SECTION 5: Inference

- Find Best Checkpoint
- Generate Speech
- Batch Inference


In [ ]:
# # Find Best Checkpoint
# import glob
# import re

# # Look for specific checkpoint patterns (avoid intermediate/corrupted files)
# best_model = glob.glob(f"{RUN_DIR}/best_model*.pth")
# checkpoints = glob.glob(f"{RUN_DIR}/checkpoint_*.pth")

# # Combine and sort all valid checkpoints
# all_checkpoints = best_model + checkpoints

# if not all_checkpoints:
#     # Fallback to any .pth file
#     all_checkpoints = glob.glob(f"{RUN_DIR}/*.pth")
#     if not all_checkpoints:
#         raise FileNotFoundError(f"❌ No checkpoints found in {RUN_DIR}")
#     print("⚠️  No standard checkpoints found, using fallback")

# # Sort by modification time (most recent first)
# all_checkpoints.sort(key=lambda x: os.path.getmtime(x), reverse=True)

# print(f"✅ Found {len(all_checkpoints)} checkpoint(s):\n")
# for i, ckpt in enumerate(all_checkpoints, 1):
#     size = os.path.getsize(ckpt) / (1024*1024)
#     basename = os.path.basename(ckpt)
#     # Extract step number if available
#     match = re.search(r'(\d+)', basename)
#     step = f"step {match.group(1)}" if match else "unknown step"
#     print(f"   {i}. {basename} ({size:.0f} MB, {step})")

# # Prefer best_model, otherwise use most recent checkpoint
# if best_model:
#     BEST = best_model[0]
#     print(f"\n🎯 Using best model: {os.path.basename(BEST)}")
# else:
#     BEST = all_checkpoints[0]
#     print(f"\n🎯 Using most recent checkpoint: {os.path.basename(BEST)}")

In [ ]:
# # Generate Speech
# from IPython.display import Audio, display

# TEXT = "Xin chào, đây là giọng nói tiếng Việt sau khi fine-tune."
# OUTPUT = f"{RUN_DIR}/inference_sample.wav"

# print(f"Synthesizing: '{TEXT}'\n")

# !python -m TTS.bin.synthesize \
#   --text "{TEXT}" \
#   --model_path "{BEST}" \
#   --config_path "{CFG_PATH}" \
#   --out_path "{OUTPUT}" \
#   --language_idx "vi"

# print(f"\n✅ Generated: {OUTPUT}")
# display(Audio(OUTPUT))

In [ ]:
# # Batch Inference
# tests = [
#     "Trí tuệ nhân tạo đang phát triển rất nhanh.",
#     "Hôm nay thời tiết đẹp, chúng ta đi dạo nhé.",
#     "Công nghệ ngày càng tiến bộ và hiện đại."
# ]

# print("Batch inference...\n")

# for i, text in enumerate(tests, 1):
#     out = f"{RUN_DIR}/test_{i}.wav"
#     print(f"{i}. {text}")

#     !python -m TTS.bin.synthesize \
#       --text "{text}" \
#       --model_path "{BEST}" \
#       --config_path "{CFG_PATH}" \
#       --out_path "{out}" \
#       --language_idx "vi" \
#       2>/dev/null

#     if os.path.exists(out):
#         display(Audio(out))

# print("\n✅ Batch complete!")